# NN Regression on Franke Function using Pytorch 

#### Import libraries

In [1]:
import sys
import os

# Add the 'src' directory to the Python path
src_dir = os.path.join(os.path.dirname(os.getcwd()), 'src')
sys.path.append(src_dir)

In [2]:
import torch

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from torch import nn, optim, tensor
from Neural_Network_with_PyTorch import Neural_Network_PyTorch

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import linear_model

from tqdm import tqdm
from copy import deepcopy

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

## Create Data

In [3]:
def FrankeFunction(x,y):
    
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

In [26]:
N = 1000
x = np.random.uniform(0, 1, N)
y = np.random.uniform(0, 1, N)

# Franke function with added noise:
z = FrankeFunction(x, y) + np.random.normal(0, 0.1, x.shape)
target = torch.tensor(z).reshape(len(z), 1)

# Making the design matrix:
X = torch.from_numpy(np.stack((x, y), axis=-1)).float()

# Splitting the data into training and test set:
test_size = 0.2
X_train, X_test, target_train, target_test = train_test_split(X, target, test_size=test_size)

X 

tensor([[0.5400, 0.6865],
        [0.3597, 0.8124],
        [0.4005, 0.8476],
        ...,
        [0.6594, 0.7230],
        [0.2349, 0.2420],
        [0.4470, 0.9988]])

## Tune model using K-fold cross validation

In [15]:
def train_with_crossval(train_X, train_target, 
                        n_features, n_hidden_layers, n_hidden_nodes, n_outputs, 
                        activation_hidden, 
                        learning_rate, l2_reg, 
                        num_epochs, n_minibatches):
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)
    # Initialize the evaluation metrices
    mse_crossval = []
    r2_crossval = []
    mse_train_crossval = []
    r2_train_crossval = []
    # Loop over the folds
    for train_idx, val_idx in kf.split(train_X):
        # Extract training and validation data
        X_train, target_train = train_X[train_idx], train_target[train_idx]
        X_val, target_val = train_X[val_idx], train_target[val_idx]
        
        # Create new network
        ffnn = Neural_Network_PyTorch(n_features, 
                                      n_hidden_layers, n_hidden_nodes, 
                                      n_outputs, 
                                      activation_function_hidden_layers=activation_hidden, 
                                      activation_function_output_layer=None)
        
        # Loss function
        criterion = nn.MSELoss()
        # Optimizer
        optimizer = optim.Adam(ffnn.parameters(), lr=learning_rate, weight_decay=l2_reg)
        
        # Train the network            
        ffnn = ffnn.train_network(X_train, target_train, 
                                  optimizer, criterion,
                                  num_iter=num_epochs, n_minibatches=n_minibatches)
        # Evaluate the network
        # Train performance
        target_train_pred = ffnn.feed_forward(X_train)
        mse_train_crossval.append(float(criterion(target_train_pred, target_train)))
        r2_train_crossval.append(r2_score(target_train, target_train_pred.detach().numpy()))
        # Validation performance
        ffnn.eval() # set model in evaluation mode
        with torch.no_grad(): # disable gradient computation
            target_val_pred = ffnn.feed_forward(X_val)
            mse_val_cv = float(criterion(target_val_pred, target_val))
            mse_crossval.append(mse_val_cv)
            r2_crossval.append(r2_score(target_val, target_val_pred))
    
    return ffnn, mse_crossval, r2_crossval, mse_train_crossval, r2_train_crossval

### Initial tuning of learning rate

Define model

In [22]:
n_hidden_layers = 2
n_hidden_nodes = 8
n_outputs = 2
n_features = X_train.shape[1]
activation_hidden = "sigmoid"
learning_rate_space = np.logspace(-4, 0, 5)
lmbd = 0.0001
n_epochs = 100
n_minibatches = 16

In [23]:
# Initialize the evaluation metrices
mse = np.zeros((len(learning_rate_space), 1))
r2 = np.zeros((len(learning_rate_space), 1))
mse_train = np.zeros((len(learning_rate_space), 1))
r2_train = np.zeros((len(learning_rate_space), 1))
for i, learning_rate in enumerate(learning_rate_space):
    print(f"Learning rate: {learning_rate}")
    
    train_with_crossval(X_train, target_train, 
                        n_features, n_hidden_layers, n_hidden_nodes, n_outputs, 
                        activation_hidden, 
                        learning_rate, lmbd, 
                        n_epochs, n_minibatches)
    
    mse[i][j] = np.mean(mse_crossval)
    r2[i][j] = np.mean(r2_crossval)
    mse_train[i][j] = np.mean(mse_train_crossval)
    r2_train[i][j] = np.mean(r2_train_crossval)

Learning rate: 0.0001
<function Neural_Network_PyTorch.set_activation_function.<locals>.dummy at 0x000001AB37645BC0>


D:\Python\envs\DataAnalysis\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: Found dtype Double but expected Float

## Retrain model on entire training set

## Estimate generalization error